In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, fbeta_score, confusion_matrix, r2_score
from datetime import date
import time

import numpy as np
from statistics import mean
from sklearn.impute import KNNImputer
from sklearn import preprocessing

from pyAgrum.lib.bn2roc import showROC_PR

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [3]:
import pyAgrum as gum

In [4]:
import pyAgrum.skbn as skbn
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.explain as exp

In [5]:
_df = pd.read_excel('20230705_extract_datacraft_v2.xlsx', index_col=0)

In [6]:
df = _df.copy()

In [7]:
#profile = ProfileReport(df, title='Pandas profiling before data preprcessing', minimal=True, progress_bar=False)
#profile.to_notebook_iframe()

# Data preparation

## See uniques values for each column

In [8]:
for col in df.columns :
    print(df[col].value_counts(dropna=False))
    print('\n')

mont_remb_init
250.00    9417
290.00    3131
180.00    1562
110.00     831
430.00     374
          ... 
254.15       1
113.68       1
547.61       1
462.29       1
233.66       1
Name: count, Length: 6683, dtype: int64


nb_avr_typ_a
0     17391
1      7928
2      1070
3       226
4        54
5        27
6        16
8        10
7         9
9         5
10        3
73        1
12        1
11        1
Name: count, dtype: int64


nb_avr_typ_b
0    26339
1      331
2       46
3       13
4        8
5        2
6        1
8        1
7        1
Name: count, dtype: int64


nb_avr_typ_c
1     16224
0      7296
2      2730
3       357
4        68
5        30
6        16
10        3
12        2
16        2
9         2
8         2
18        2
15        2
7         1
21        1
24        1
11        1
20        1
22        1
Name: count, dtype: int64


nb_avr_typ_d
0     26277
1       416
2        40
3         4
4         3
38        1
5         1
Name: count, dtype: int64


sum_mont_avr
-250    78

## Feature MONT_REMB_INIT

In [9]:
#PRETRAITEMENT PARTICULIER MONT REMB INIT
col_remb = df['mont_remb_init']
col_remb = col_remb.replace(250, '250')
col_remb = col_remb.replace(180, '180')
col_remb = col_remb.replace(290, '290')
col_remb.loc[col_remb.isin(['250', '180', '290'])==False] = pd.qcut(
    col_remb.loc[col_remb.isin(['250', '180', '290'])==False].astype('float64'), 
    5, labels = ['100-114','114-150','150-220', '220-344', '344-3300' ], duplicates='drop')

df['mont_remb_init'] = col_remb

In [10]:
df['mont_remb_init'].value_counts()

mont_remb_init
250         9417
290         3131
100-114     2535
344-3300    2527
220-344     2526
150-220     2523
114-150     2521
180         1562
Name: count, dtype: int64

## Drop column with only one value or column with 2 differents values but less than 5 occurences

In [11]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        print(col)
        df = df.drop(col,axis=1)
        
    elif len(df[col].unique()) == 2 :
        if np.any(df[col].value_counts().values<5) :
            print(col)
            #print(df[col].value_counts())
            df = df.drop(col,axis=1)

is_cod_comp_3
is_cod_comp_12
is_cod_comp_16
is_cod_comp_26
is_cod_comp_34
is_cod_comp_35
is_cod_comp_59
is_cod_comp_79
is_cod_comp_84
is_cod_comp_87
is_cod_comp_88
is_rendu_A
is_rendu_D
is_rendu_G
is_attendu_A
is_attendu_G
is_evnm_decl_5
nb_ssc_typ_2
nb_ssc_typ_3
nb_resil_2
nb_resil_3
nb_mig_typ_2
nb_mig_typ_3
is_achat_com_typ_9
is_achat_com_typ_13
is_achat_lig_typ_12
is_ig_service
flg_op_2
flg_fut_cds


## PRETRAITEMENT PARTICULIER sum_mont_avr

In [12]:
#PRETRAITEMENT PARTICULIER sum_mont_avr
col_sum = df['sum_mont_avr']
col_sum = col_sum.replace(-250, '-250')
col_sum = col_sum.replace(-180, '-180')
col_sum = col_sum.replace(-290, '-290')
col_sum.loc[col_sum.isin(['-250', '-180', '-290'])==False] = pd.qcut(
    col_sum.loc[col_sum.isin(['-250', '-180', '-290'])==False].astype('float64'), 
    5, labels = ['-3562,-382','-382,-289','-289,-204', '-204,-132', '-132,0'], duplicates='drop')

df['sum_mont_avr'] = col_sum

In [13]:
df['sum_mont_avr'].value_counts()

sum_mont_avr
-250          7856
-289,-204     3022
-204,-132     3016
-382,-289     3002
-3562,-382    2999
-132,0        2946
-290          2728
-180          1173
Name: count, dtype: int64

## Regroup less frequent values together

In [14]:
def orphan_values(col, min_value) :
    df[col] = df[col].replace(sorted(df[col].unique())[min_value:], str(min_value)+'_plus')
    df[col] = df[col].astype('str')

In [15]:
orphan_values('nb_eqpt_rendu', 5)

In [16]:
orphan_values('nb_appels', 23)

In [17]:
orphan_values('nb_lig_mob_pu', 5)

In [18]:
orphan_values('nb_lig_mob_fgr',5)

In [19]:
orphan_values('nb_lig_fix_pu',4)

In [20]:
orphan_values('nb_chgt_rib',1)

In [21]:
df['nb_chgt_rib'].value_counts()

nb_chgt_rib
0         26663
1_plus       79
Name: count, dtype: int64

In [22]:
df = df.loc[df['soc_cons']!='C']

## Replace Nan with specific number

In [23]:
df['numr_cycl_fact'] = df['numr_cycl_fact'].replace(np.nan, -1)
df['numr_cycl_fact'].value_counts(dropna=False)

numr_cycl_fact
 1.0     3841
 8.0     3653
 7.0     3322
 4.0     3242
 6.0     2851
 5.0     2821
 3.0     2585
 2.0     2117
-1.0     1398
 17.0     144
 31.0     131
 12.0     128
 0.0      119
 33.0     119
 34.0      33
 38.0      32
 14.0      27
 11.0      26
 37.0      25
 18.0      25
 15.0      21
 16.0      20
 13.0      16
 32.0      16
 35.0      15
 36.0      14
Name: count, dtype: int64

In [24]:
df['idnt_motf_piec_comp'] = df['idnt_motf_piec_comp'].replace('na', -1)
df['idnt_motf_piec_comp'] = df['idnt_motf_piec_comp'].astype('int32')
df['idnt_motf_piec_comp'].value_counts(dropna=False)

idnt_motf_piec_comp
-1     18787
 30     1934
 34     1910
 10      991
 6       945
 9       743
 8       371
 35      232
 23      147
 11      121
 31       89
 7        79
 18       53
 22       51
 40       49
 20       40
 33       38
 32       29
 21       25
 50       24
 55       23
 56       15
 51       14
 41        6
 37        6
 15        3
 36        3
 27        3
 19        2
 53        2
 75        1
 45        1
 84        1
 24        1
 46        1
 70        1
Name: count, dtype: int64

In [25]:
df['idnt_motf_piec_comp'] = pd.qcut(df['idnt_motf_piec_comp'], 5, labels = ['-1','autre'], duplicates='drop')

In [26]:
df['idnt_motf_piec_comp']

id
1           -1
2           -1
3           -1
4           -1
5        autre
         ...  
26738    autre
26739       -1
26740       -1
26741       -1
26742       -1
Name: idnt_motf_piec_comp, Length: 26741, dtype: category
Categories (2, object): ['-1' < 'autre']

## Quantile-based discretization 

In [27]:
df['sum_prx_offr_bse_arr_ttc'] = pd.qcut(df['sum_prx_offr_bse_arr_ttc'], 5, labels=False)

In [29]:
#df['anc_cp_jours'] = pd.qcut(df['anc_cp_jours'], 5, labels=False)

In [30]:
df['sum_mmf3_cds'] = pd.qcut(df['sum_mmf3_cds'], 5, labels=False,duplicates='drop')

In [31]:
df['sum_dur_crdt'].loc[df['sum_dur_crdt']!=0] = 1

/var/folders/m7/x040g7pj7rgg18j8yj5b14gw0000gn/T/ipykernel_13868/3528659070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sum_dur_crdt'].loc[df['sum_dur_crdt']!=0] = 1


In [32]:
df['sum_mont_crdt'].loc[df['sum_mont_crdt']!=0] = 1

/var/folders/m7/x040g7pj7rgg18j8yj5b14gw0000gn/T/ipykernel_13868/2312795950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sum_mont_crdt'].loc[df['sum_mont_crdt']!=0] = 1


In [33]:
df['anc_dernier_repri'].loc[df['anc_dernier_repri']!=0] = 1

/var/folders/m7/x040g7pj7rgg18j8yj5b14gw0000gn/T/ipykernel_13868/3231036602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['anc_dernier_repri'].loc[df['anc_dernier_repri']!=0] = 1


In [34]:
df['anc_dernier_cibl_enrichi'].loc[df['anc_dernier_cibl_enrichi']!=0] = 1

/var/folders/m7/x040g7pj7rgg18j8yj5b14gw0000gn/T/ipykernel_13868/258784937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['anc_dernier_cibl_enrichi'].loc[df['anc_dernier_cibl_enrichi']!=0] = 1


In [35]:
df['anc_dernier_refus_enrichi'].loc[df['anc_dernier_refus_enrichi']!=0] = 1

/var/folders/m7/x040g7pj7rgg18j8yj5b14gw0000gn/T/ipykernel_13868/415494574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['anc_dernier_refus_enrichi'].loc[df['anc_dernier_refus_enrichi']!=0] = 1


In [36]:
df = df.drop(list(df.columns[df.columns.str.contains('sum_achat_typ')]), axis=1)

### Retrieve columns where there are values with less than 5 occurrences. 

In [37]:
cols = []

for col in df.columns:
    if np.any(df[col].value_counts().values<5) :
        print(col)
        cols.append(col)

nb_avr_typ_a
nb_avr_typ_b
nb_avr_typ_c
nb_avr_typ_d
nb_pa_rjt
nb_eqpt_attendu
nb_ssc_typ_1
nb_resil_1
nb_resil_4
nb_mig_typ_1
nb_mig_typ_4
nb_impayes
nb_act_vent
nb_ig
nb_lig_fix_fgr
nb_repri
nb_cibl_enrichi
nb_enrichi_refus


In [38]:
def is_sum_grd(data,i=2):
    if data.value_counts()[i:].sum() > data.value_counts()[i-1]:
        is_sum_grd(data,i+1)
    else :
        print(data.value_counts().index[i])
        return data.value_counts().index[i]

### Find minimal value for where to regroup

In [39]:
for col in cols :
    print(col)
    mn = is_sum_grd(df[col])
    orphan_values(col, mn)

nb_avr_typ_a
2
nb_avr_typ_b
2
nb_avr_typ_c
2
nb_avr_typ_d
2
nb_pa_rjt
2
nb_eqpt_attendu
4
nb_ssc_typ_1
2
nb_resil_1
2
nb_resil_4
2
nb_mig_typ_1
2
nb_mig_typ_4
2
nb_impayes
2
nb_act_vent
2
nb_ig
2
nb_lig_fix_fgr
2
nb_repri
2
nb_cibl_enrichi
2
nb_enrichi_refus
2


In [40]:
df[['mont_remb_init', 'sum_mont_avr']].loc[df['cible']==1]

,mont_remb_init,sum_mont_avr
id,,
4,220-344,"-382,-289"
8,290,-290
11,250,"-382,-289"
18,114-150,"-204,-132"
21,220-344,"-382,-289"
...,...,...
26720,250,-250
26722,290,-290
26723,250,"-289,-204"


In [41]:
df[['mont_remb_init', 'sum_mont_avr']].loc[df['cible']==0]

,mont_remb_init,sum_mont_avr
id,,
1,290,-290
2,100-114,"-204,-132"
3,100-114,"-132,0"
5,220-344,"-289,-204"
6,250,-250
...,...,...
26737,114-150,"-132,0"
26739,250,"-289,-204"
26740,250,-250


In [42]:
df['cible'].value_counts(dropna=False)

cible
0    22904
1     3837
Name: count, dtype: int64

In [43]:
df

,mont_remb_init,nb_avr_typ_a,nb_avr_typ_b,nb_avr_typ_c,nb_avr_typ_d,sum_mont_avr,nb_pa_rjt,typ_avr_decl,is_cod_comp_1,is_cod_comp_2,...,max_flg_pro_cds,flg_coll,sum_mmf3_cds,nb_repri,anc_dernier_repri,nb_cibl_enrichi,anc_dernier_cibl_enrichi,nb_enrichi_refus,anc_dernier_refus_enrichi,cible
id,,,,,,,,,,,,,,,,,,,,,
1,290,0,0,1,0,-290,0,A,0,0,...,0,0,0,0,1,0,1,0,1,0
2,100-114,1,0,0,0,"-204,-132",2_plus,B,0,0,...,0,0,0,0,1,0,1,0,1,0
3,100-114,0,0,1,0,"-132,0",0,A,0,0,...,0,0,0,0,1,0,1,0,1,0
4,220-344,1,0,0,0,"-382,-289",0,B,0,0,...,0,0,1,0,1,1,1,1,1,1
5,220-344,2_plus,0,0,0,"-289,-204",0,B,0,0,...,0,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26738,220-344,1,0,0,0,"-289,-204",0,B,0,0,...,0,0,1,0,1,1,1,0,1,1
26739,250,0,0,2_plus,0,"-289,-204",0,A,0,0,...,0,0,0,0,1,0,1,0,1,0
26740,250,0,0,1,0,-250,0,A,0,0,...,0,0,0,0,1,0,1,0,1,0


In [44]:
def cutqcut(df, nb = 30, nb_bins=10) :
    for col in df.columns:
        if len(df[col].unique()) > nb:
            if df[col].dtype != 'object' :
                print(col)
                print(len(df[col].unique()))
                df[col] = pd.cut(df[col], nb_bins, labels=False, duplicates='drop') #

    return df

In [45]:
cutqcut(df, nb=9, nb_bins=9)

last_ig_crtn
182
numr_cycl_fact
26


,mont_remb_init,nb_avr_typ_a,nb_avr_typ_b,nb_avr_typ_c,nb_avr_typ_d,sum_mont_avr,nb_pa_rjt,typ_avr_decl,is_cod_comp_1,is_cod_comp_2,...,max_flg_pro_cds,flg_coll,sum_mmf3_cds,nb_repri,anc_dernier_repri,nb_cibl_enrichi,anc_dernier_cibl_enrichi,nb_enrichi_refus,anc_dernier_refus_enrichi,cible
id,,,,,,,,,,,,,,,,,,,,,
1,290,0,0,1,0,-290,0,A,0,0,...,0,0,0,0,1,0,1,0,1,0
2,100-114,1,0,0,0,"-204,-132",2_plus,B,0,0,...,0,0,0,0,1,0,1,0,1,0
3,100-114,0,0,1,0,"-132,0",0,A,0,0,...,0,0,0,0,1,0,1,0,1,0
4,220-344,1,0,0,0,"-382,-289",0,B,0,0,...,0,0,1,0,1,1,1,1,1,1
5,220-344,2_plus,0,0,0,"-289,-204",0,B,0,0,...,0,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26738,220-344,1,0,0,0,"-289,-204",0,B,0,0,...,0,0,1,0,1,1,1,0,1,1
26739,250,0,0,2_plus,0,"-289,-204",0,A,0,0,...,0,0,0,0,1,0,1,0,1,0
26740,250,0,0,1,0,-250,0,A,0,0,...,0,0,0,0,1,0,1,0,1,0


In [46]:
for col in df.columns :
    print(df[col].value_counts(dropna=False))
    print('\n')
    

mont_remb_init
250         9417
290         3131
100-114     2535
344-3300    2527
220-344     2525
150-220     2523
114-150     2521
180         1562
Name: count, dtype: int64


nb_avr_typ_a
0         17391
1          7927
2_plus     1423
Name: count, dtype: int64


nb_avr_typ_b
0         26338
1           331
2_plus       72
Name: count, dtype: int64


nb_avr_typ_c
1         16224
0          7295
2_plus     3222
Name: count, dtype: int64


nb_avr_typ_d
0         26276
1           416
2_plus       49
Name: count, dtype: int64


sum_mont_avr
-250          7856
-289,-204     3022
-204,-132     3016
-382,-289     3001
-3562,-382    2999
-132,0        2946
-290          2728
-180          1173
Name: count, dtype: int64


nb_pa_rjt
0         21354
1          3745
2_plus     1642
Name: count, dtype: int64


typ_avr_decl
A    18420
B     8001
C      180
D      130
E       10
Name: count, dtype: int64


is_cod_comp_1
0    26724
1       17
Name: count, dtype: int64


is_cod_comp_2
0    25011
1

### Save to CSV

In [47]:
df.to_csv('df_pretreat_v2.csv', index=False)

In [47]:
f"{3.50:g}"

'3.5'

In [48]:
f"{2.00:g}"

'2'

In [49]:
f"{3.505:g}"

'3.505'

In [50]:
f"{300:g}"

'300'

In [51]:
f"{30.00:g}"

'30'